# TmVO4 neutrons data analysis
Fit neutrons diffraction peaks measured on TmVO4 at SNS on 2019-02-14 in order to extract the orthorhombic distortion as a function of magnetic field

## Import modules

In [64]:
from mpl_toolkits.mplot3d import Axes3D# for 3D plotting
import matplotlib.tri as mtri# for triangulation of unevenly separated data, like our magnetic field data

import copy as cp, numpy as np, pandas as pd, pickle, os, re
import importlib, sys 
import matplotlib
from matplotlib import cm, pyplot as plt, rcsetup, rc, rcParams# import matplotlib.pyplot as plt
# cm stands for colormap
from matplotlib.ticker import LogLocator, LinearLocator, FormatStrFormatter
from scipy.interpolate import griddata
from scipy.special import erfc, exp1

from lmfit import minimize, Model, Parameters, report_fit, fit_report
# import pytest

### Change default plotting parameters
see https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.figure.html

In [28]:
rcParams["figure.figsize"] = np.multiply([6.4, 4.8],0.5)# default is [6.4, 4.8]

## Import data

### Coarse data

In [30]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14'
os.chdir(tempPath)
(_, _, filenames) = next(os.walk(tempPath))# the walk() function lists the content of the directory that it is given as argument,
# and of its subdirectories; the next() function returns the next output of the walk() function;
# when used only once, it returns only the first output which is the content of the parent directory, 
# listed as a tuple of the form (dirpath, dirnames, filenames)
filenames = [filename for filename in filenames if 'p6K_' in filename]# keep only files that match the string pattern of datafilestempList = [None]*len(filenames)# Pre-allocate temporary list to store the data that will then be converted into a Pandas DataFrame

tempList = [None]*len(filenames)
for idx, filename in enumerate(filenames):# For each data file
    H = float(re.split('p6K_(\w*)T\w*.txt',filenames[idx])[1].replace('p','.'))# Extract value of magnetic field from filename
    tempDF = pd.read_csv(filenames[idx],names=["hh0","I","dI"],skiprows=2,delimiter=',')# import data as a Pandas DataFrame
    tempList[idx] = {'filename': filenames[idx],# Store into a dictionary, which is itself an element of tempList: the filename,
                   'H (T)': H,# value of magnetic field,
                   'T (K)': 0.6,# temperature,
                   'spectra': tempDF# and data stored as Pandas dataframe
                   }
coarseData = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
del idx, H, tempPath, filename, filenames, tempList, tempDF# delete temporary variables after use 
coarseData# Show the resulting DataFrame

,filename,H (T),T (K),spectra
0,p6K_1T.txt,1.00,0.6,hh0 I dI 0 -11.99...
1,p6K_1T_new.txt,1.00,0.6,hh0 I dI 0 -11.99...
2,p6K_p05T.txt,0.05,0.6,hh0 I dI 0 -11.9975...
3,p6K_p4T.txt,0.40,0.6,hh0 I dI 0 -11.997...
4,p6K_p5T.txt,0.50,0.6,hh0 I dI 0 -11.99...
5,p6K_p6T.txt,0.60,0.6,hh0 I dI 0 -11.99...
6,p6K_p77T.txt,0.77,0.6,hh0 I dI 0 -11.99...
7,p6K_p7T.txt,0.70,0.6,hh0 I dI 0 -11.99...
8,p6K_p83T.txt,0.83,0.6,hh0 I dI 0 -11.99...
9,p6K_p97T.txt,0.97,0.6,hh0 I dI 0 -11.99...


In [31]:
coarseData.spectra[3].head()# Check the content of individual datasets after importation

,hh0,I,dI
0,-11.9975,142028.0,52309.7
1,-11.9925,220372.0,62925.4
2,-11.9875,220002.0,62818.9
3,-11.9825,518428.0,92939.8
4,-11.9775,817317.0,127501.0


### Fine data

In [32]:
tempPath = r'C:\Users\Pierre\Desktop\Postdoc\TmVO4\TmVO4_neutrons\2019-02_ORNL_Corelli\2019-02-14\p6K\linecut_f'
os.chdir(tempPath)
fieldInfo = pd.read_csv('field_info.txt',header=None,names=['File #','T (K)','H (T)','Proton charge'],delimiter=' ')
fieldInfo['File #'] = fieldInfo['File #'].astype('int')# replace file number type from float to int
fieldInfo.head()
# fieldInfo['File #'].dtype# check that the change of datatype is effective

,File #,T (K),H (T),Proton charge
0,88631,0.605973,0.000000,1.312139
1,88632,0.613334,0.049998,0.800119
2,88633,0.605727,0.099996,0.083534
3,88634,0.625493,0.099996,0.800739
4,88635,0.618220,0.150005,0.801294


In [33]:
tempList = [None]*len(fieldInfo)# Preallocate list to store the data that will then be converted into a Pandas DataFrame
for idx in range(len(fieldInfo)):# For each data file
    filename = ''.join(['HH0_',str(fieldInfo['File #'][idx]),'.txt'])
#     print(filename)
    tempDF = pd.read_csv(filename,names=["hh0","I","dI"],skiprows=2,delimiter=',')# Import data as a Pandas DataFrame
    tempList[idx] = {'filename': filename,# Store into a dictionary, which is itself an element of dfList: the filename,
                     'T (K)': fieldInfo['T (K)'][idx],# temperature,
                     'H (T)': fieldInfo['H (T)'][idx],# value of magnetic field,
                     'Proton charge': fieldInfo['Proton charge'][idx],# proton charge,
                     'spectra': tempDF# and data,
                     }
linecut_f_raw = pd.DataFrame(tempList)# Convert the list of dictionaries into a Pandas DataFrame
linecut_f_raw.head()# Show the resulting DataFrame

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


In [34]:
del idx, tempPath, filename, tempList, tempDF# delete temporary variables after use 

In [35]:
linecut_f_raw['spectra'][3].loc[180:185]#.head()# Check the content of individual datasets after importation

,hh0,I,dI
180,-12.5487,0.0,0.0
181,-12.5462,0.0,0.0
182,-12.5437,0.0,0.0
183,-12.5412,0.0,0.0
184,-12.5388,0.0,0.0
185,-12.5363,0.0,0.0


## Basic data processing
Rescale data and remove "bad" data

### Check consistency of dataframes

#### <a name="hh0_consistency"></a>Check that all hh0 data are the same within each dataframe
i.e. that all hh0 of coarseData are the same and that all hh0 data of linecut_f_raw are the same

In [37]:
nData_raw = [coarseData,linecut_f_raw]
for data_idx in range(len(nData_raw)):# for each dataset
    for _, row in nData_raw[data_idx].iterrows():# loop over all rows
        if not np.array_equal(row.spectra.hh0,nData_raw[data_idx].spectra[0].hh0):
        # and compare the array of hh0 of that row with that of the first row
            print(row)# print the row if the two arrays are *not* equal
            # should output nothing, which means that all arrays of hh0 are the same *within a dataset*

#### Then check that hh0 data of coarseData differ from that of linecut_f_raw
We do not need to loop over all rows since we have shown in the previous cell that all rows are the same within a dataset

In [38]:
if not np.array_equal(coarseData.spectra[0].hh0,linecut_f_raw.spectra[0].hh0):
# compare the hh0 arrays of the first row of both datasets
    print("The arrarys of hh0 are not the same in both datasets")

The arrarys of hh0 are not the same in both datasets


### Which datasets to analyze and how? 

#### Update 2020-04-06
In fact, ignore the coarse dataset, as it will require a lot of efforts for a minimal result.
More interesting would be to analyze the linecut_f dataset at each (hh0) peak position, for h = 6, 8, 10.

##### Update 2020-04-03
treat both datasets independently in terms of the plotting and fitting

##### Outdated ideas
involving treating both datasets together, which will make things more complicated, and therefore increases the risks of making errors, in addition to increasing the time required for the analysis:
* interpolate spectra of coarseData so that its hh0 array is the same as that of linecut_f_raw
* rescale the data, if there is a physical way to do it, otherwise simply treat both datasets separately

### Truely "deep" copy of linecut_f
such that linecut_f_raw will *not* be modified if lcf_copy is modified

The best way to truely, i.e. recursively, deep copy a python object is to `pickle.dump` and `pickle.load` it. 
That is because `cPickle` is the fastest, as shown [here](https://stackoverflow.com/questions/1410615/copy-deepcopy-vs-pickle),  *and* **in Python 3**, `cPickle` is the default behavior of `pickle`, as explained [here](https://askubuntu.com/a/804618).
See also hacks.ipynb, and [here](https://stackoverflow.com/questions/52708341/make-a-truly-deep-copy-of-a-pandas-series). 

In [39]:
lcf_copy = pickle.loads(pickle.dumps(linecut_f_raw))# fastest python hack to create a truely deep copy
# lcf stands for linecut_f, which is the name of the file from which the data was imported
# idx = 5
lcf_copy.head()#.loc[idx:idx+10]

,filename,T (K),H (T),Proton charge,spectra
0,HH0_88631.txt,0.605973,0.000000,1.312139,hh0 I dI 0 -12.99880 0.0 ...
1,HH0_88632.txt,0.613334,0.049998,0.800119,hh0 I dI 0 -12.99880 0.0 ...
2,HH0_88633.txt,0.605727,0.099996,0.083534,hh0 I dI 0 -12.99880 0.0 ...
3,HH0_88634.txt,0.625493,0.099996,0.800739,hh0 I dI 0 -12.99880 0.0 ...
4,HH0_88635.txt,0.618220,0.150005,0.801294,hh0 I dI 0 -12.99880 0.0 ...


### Data rescaling
This used to be done manually, after noticing that the intensity of the data at 0T was higher than that of the rest of the data. A factor of 0.635 was then used to rescale this spectrum to the level of other data. 
However, after getting the up-to-date data, it appears that the scaling factor is merely the proton charge, which is a proxy for the counting time of neutrons. With this information, it turns out that the ratio of Proton charges of the spectrum at 0T and that at 0.05T is 1.3 to 0.8. The rescaling factor of the former is thus 0.8/1.3=0.615. Hence the empirical value of 0.635 was a pretty good guess!

In [40]:
##### Rescale data according to their Proton charge
for idx, row in lcf_copy.iterrows():
    row.spectra['Inorm'] = row.spectra.I/row['Proton charge']
    row.spectra['dInorm'] = row.spectra.dI/row['Proton charge']
#     print(idx, row['spectra']['I'])
idx = 1200# index that allows to look at data close to the (10 10 0) peak
lcf_copy.spectra[0].loc[idx:idx+5]# check one of the resulting dataframes

,hh0,I,dI,Inorm,dInorm
1200,-9.99875,1261600.0,94684.7,9.614833e+05,72160.557743
1201,-9.99625,1230390.0,98819.2,9.376977e+05,75311.519049
1202,-9.99375,1448420.0,109920.0,1.103862e+06,83771.596753
1203,-9.99125,2141070.0,124810.0,1.631740e+06,95119.477718
1204,-9.98875,2560080.0,141466.0,1.951073e+06,107813.252422
1205,-9.98625,4044660.0,169302.0,3.082493e+06,129027.464278


### Find bad data, if any
#### Identify datasets with zero intensity and create a new "clean" dataset without bad data

In [41]:
lcf_clean = lcf_copy.copy()
# this is a deepcopy according to Pandas, which is only deep at the lowest order, i.e. not recursively
for idx, row in lcf_copy.iterrows():# loop over all spectra
    if not np.any(row.spectra.Inorm>0):# if any spectrum has a constant zero intensity
        lcf_clean = lcf_copy.drop(idx)
        print(idx)# output the row index of lcf_copy containing empty data
# lcf_copy.spectra[0].Inorm

84


#### Batch plot the rest of the spectra to identify other potential bad data

In [42]:
%matplotlib qt
# plot figures in external window

##### Plot spectra by groups of nSpec
to see if any spectrum differs from the others.
This method may fail if there are more than nSpec bad consecutive spectra. Hence nSpec should not be too small (5 or more should be good)

In [43]:
nSpec = 7# number of curves in each plot
nfig = len(lcf_clean)//nSpec+1
fig = [None]*(nfig)
lgd = [None]*(nfig)
for fidx in range(1):#range(nfig):
    fig[fidx] = plt.figure()
    ax = fig[fidx].add_subplot(1,1,1)
    for pidx in range(nSpec*fidx,nSpec*(fidx+1)):
        try:
            plt.plot(lcf_clean.spectra[pidx].hh0,lcf_clean.spectra[pidx].Inorm,\
                     label=f"{pidx}, {lcf_clean['H (T)'][pidx]:.2f}")
        except KeyError:# if the index of data to be plotted does not exist
            continue# ignore and carry on to the next one
    plt.xlim(-10.25,-5.75)
    lgd[fidx] = ax.legend(title='Index, H (T)')
    lgd[fidx].set_draggable(True)
    plt.show()# ensures that all windows come to the foreground

##### Notes --- 2020-04-07
* After plotting all spectra together, it appears that only spectra #2 and #85 are "bad": the former is very noisy (not too surprising given that its Proton charge is 1/10 of the other data) and the latter has a lower intensity than the other spectra (for an unknown reason).
* Both those bad spectra were identified during the measurement, such that another (good) spectrum was measured at both values of magnetic field at which those bad spectra had been measured. Concretely, spectrum #2 was measured at 0.1T and #85 at 0.865T but are bad. Spectra #3 and #104 were also measured at 0.1T and 0.865T, respectively, and are good. Hence, the two bad spectra should just be discarded, there is no reason to try and process them in order to try and make them good: first because there is no way of making good data out of bad data (bad data is just bad data), so it would be a waste of time, and potentially a lot of time, but it would also be complicated, and because the result cannot be good, it can only influence the result of the subsequent fits in a bad way, thus inducing distrust on the results obtained over the whole dataset instead of just two spectra. Bottom line: discard spectra #2 and #85.

### Remove bad data 
as identified in batch plotting

In [44]:
delRowIdx = [2,85]# index of data to remove
lcf_clean.drop(delRowIdx,inplace=True)

### Final clean dataset
#### Sorted by value of magnetic field

In [45]:
nData = lcf_clean.sort_values(by=['H (T)'],ignore_index=True)
# lcf_clean.head()# nData stands for "neutrons Data"

#### Plot a couple of ENS spectra (ignore after done once)
to check consistency between spectra before fitting

### Plot entire dataset in 3D
as spectrum normalized intensity 'Inorm' vs position in reciprocal space 'hh0' and magnetic field 'H (T)', to check that the field dependence of the data is consistent.
#### Ignore after done once
Change cells to Raw type in order to avoid running inadvertantly

#### First plot individual spectra in a 3D space

#### Then plot 3D color map
##### Create meshgrid for 3D color map
The mesh can safely be created using hh0 data from the dataset measured at any value of magnetic field, since we've shown [that all hh0 data are the same within the linecut_f dataframe](#hh0_consistency)

##### Create the 2D array that contains the intensity data

##### Plot the hh0mesh and Hmesh data to see how irregular they are

##### Conclusion
The hh0 data looks pretty regular, however the magnetic field data is not
##### Update
That should not prevent from plotting in 3D

#### Use griddata to interpolate the intensity data over a regular array

#### Plot 3D surface
Note: Matplotlib does not do a very good job of plotting the 3D surface when the xlim is not adapted to the range of the data and when there is noise in the data: in our case, it looks like spectra are plotted individually rather than as a continuous surface. When zooming on each peak, the surface looks a little better. Perhaps look for a better 3D visualization tool.

#### Matplotlib example of 3D surface plotting

## Fit data

### Define fit result plotting function

In [19]:
def plot_fit_result(x, y, fitresult):
    """
    Plot result of fit using an lmfit Model object
    """
    fig = plt.figure()
    plt.plot(x, y, 'bo')
    plt.plot(x, fitresult.init_fit, 'k--', label='initial fit')
    plt.plot(x, fitresult.best_fit, 'r-', label='best fit')
    plt.legend(loc='best')
    plt.show()
    

#### Attempt to an iteration callback function
Does not work as of 2020-04-14

In [35]:
def iter_func(params, iter_num, resid, x):
    print(iter_num)
    return True

### Fit example adapted from the lmfit documentation 
(section "Modeling Data and Curve Fitting)

In [30]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x-cen)**2 / wid)
gmodel = Model(gaussian)
print('parameter names: {}'.format(gmodel.param_names))
print('independent variables: {}'.format(gmodel.independent_vars))
gparams = gmodel.make_params(cen=5, amp=20, wid=1)
gparams

parameter names: ['amp', 'cen', 'wid']
independent variables: ['x']


name,value,initial value,min,max,vary
amp,20.0000000,None,-inf,inf,True
cen,5.00000000,None,-inf,inf,True
wid,1.00000000,None,-inf,inf,True


In [31]:
x = np.linspace(0,10,101)
noise = np.random.rand(101)-0.5
y = gaussian(x,amp=20,cen=5,wid=1)+noise
# fig = plt.figure()
# plt.plot(x, y)
# plt.show()
result = gmodel.fit(y, x=x, amp=5, cen=5, wid=1)
print(result.fit_report())
plot_fit_result(x, y, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 17
    # data points      = 101
    # variables        = 3
    chi-square         = 8.43330605
    reduced chi-square = 0.08605414
    Akaike info crit   = -244.776096
    Bayesian info crit = -236.930735
[[Variables]]
    amp:  19.9229410 +/- 0.10155032 (0.51%) (init = 5)
    cen:  4.99454353 +/- 0.00415645 (0.08%) (init = 5)
    wid:  0.99742555 +/- 0.01174109 (1.18%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


### General parameters for our problem

In [20]:
peak_center = -8.0# center of unsplit peak to be studied in the following, in reciprocal space units
npeak_interval = .15# half of plot interval
Hc_0 = 0.51# value in Tesla units of the critical field at zero temperature
# in the absence of demagnetizing factor
# see data taken on needles of TmVO4-LS5200 in July 2017

### Single-peak fit with single pVIC function on single spectrum
measured above Hc, where the peaks are unsplit

#### Select data

In [88]:
dat_idx = 100
data_select = np.logical_and(nData.spectra[dat_idx].hh0 > peak_center - npeak_interval, 
                             nData.spectra[dat_idx].hh0 < peak_center + npeak_interval)
X = nData.spectra[dat_idx].hh0[data_select]# select data for first fit
Yfitdata = nData.spectra[dat_idx].Inorm[data_select]
X.head()

1940   -8.14875
1941   -8.14625
1942   -8.14375
1943   -8.14125
1944   -8.13875
Name: hh0, dtype: float64

#### Perform fit with Gaussian model as a first try

In [33]:
np.seterr(all='warn')# Set how floating-point errors are handled
# Setting warnings will help identify computation issues that may result in an unsuccessful fitting procedure
ngparams = gmodel.make_params(cen=peak_center, amp=4e6, wid=1e-3)
# Yeval = gmodel.eval(ngparams,x=X)# evaluate model using initial parameters
result = gmodel.fit(data=Yfitdata, params=ngparams, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)

[[Model]]
    Model(gaussian)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 33
    # data points      = 120
    # variables        = 3
    chi-square         = 2.6305e+12
    reduced chi-square = 2.2483e+10
    Akaike info crit   = 2863.28327
    Bayesian info crit = 2871.64575
[[Variables]]
    amp:  4232624.39 +/- 61044.4590 (1.44%) (init = 4000000)
    cen: -7.99724868 +/- 2.1258e-04 (0.00%) (init = -8)
    wid:  3.2588e-04 +/- 1.0854e-05 (3.33%) (init = 0.001)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, wid) = -0.577


#### Single pVIC fit model

##### Make model from fit function

In [21]:
from ENS_peak_fit_pVIC_py.pseudoVoigtIkedaCarpenter import pVIC, xpVIC_residual, xpVIC_init_prm
pvic_model = Model(pVIC)# create Model object from the lmfit module
print(f'parameter names: {pvic_model.param_names}')
print(f'independent variables: {pvic_model.independent_vars}')

parameter names: ['A', 'alpha', 'beta', 'R', 'gamma', 'sigma', 'k', 'xp']
independent variables: ['x']


#### Fit parameters
##### Create fit parameters and specify their properties
including initial values, constraints, etc.

In [110]:
pvic_params = pvic_model.make_params(A=2e5, alpha=140, beta=1e-3, R=1e-3, 
                                     gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center)
# Yfitdata = nData.spectra[0].Inorm
# pvic_params = pvic_model.guess(Yfitdata)# returns NotImplementedError
for k in pvic_params.keys():
    pvic_params[k].set(min=0, vary=True)
#     pvic_params[k].init_value = pvic_params[k].value
#     print(k)
pvic_params['xp'].set(min=-np.inf)
pvic_params['k'].vary = False
pvic_params

name,value,initial value,min,max,vary
A,200000.000,None,0.00000000,inf,True
alpha,140.000000,None,0.00000000,inf,True
beta,1.0000e-03,None,0.00000000,inf,True
R,1.0000e-03,None,0.00000000,inf,True
gamma,1.0000e-03,None,0.00000000,inf,True
sigma,0.00660000,None,0.00000000,inf,True
k,0.05000000,None,0.00000000,inf,False
xp,-8.00000000,None,-inf,inf,True


#### Evaluate fit function

In [26]:
np.seterr(all='warn')# Set how floating-point errors are handled
Yeval = pvic_model.eval(pvic_params,x=X)# evaluate model using initial parameters
fig = plt.figure()
plt.plot(X,Yeval)
plt.plot(X,Yfitdata)
plt.show()

#### Next steps as of 2020-04-14
* Play with values of beta to get best fit result; in particular, let beta vary (with max value of 800?)
* Is there a way to output the adjusted R**2 of the fit?

#### Narrowing down the number of free parameters

##### 7 free parameters
all but k

In [124]:
pvic_params['beta'].set(value=1e-10, vary=False)
pvic_params['R'].set(value=0, vary=False)
# pvic_params
result = pvic_model.fit(data=Yfitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 95
    # data points      = 120
    # variables        = 5
    chi-square         = 3.3204e+11
    reduced chi-square = 2.8873e+09
    Akaike info crit   = 2618.92465
    Bayesian info crit = 2632.86211
[[Variables]]
    A:      139800.679 +/- 1060.80779 (0.76%) (init = 200000)
    alpha:  130.275459 +/- 1.93231449 (1.48%) (init = 140)
    beta:   1e-10 (fixed)
    R:      0 (fixed)
    gamma:  2.2694e-07 +/- 1.8336e-04 (80796.03%) (init = 0.001)
    sigma:  0.00596552 +/- 2.3801e-04 (3.99%) (init = 0.0066)
    k:      0.05 (fixed)
    xp:    -8.01731207 +/- 2.3160e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, xp)    =  0.949
    C(A, gamma)     =  0.722
    C(gamma, sigma) = -0.480
    C(alpha, sigma) =  0.452
    C(sigma, xp)    =  0.444
    C(alpha, gamma) =  0.394
    C(gamma, xp)    =  0.355
    C(A, sigma)     = -0.346


The above yields too many correlations between parameters
The resulting plot also has an unphysical dip in the resulting fit function

##### 6 free parameters

In [66]:
pvic_params['beta'].set(value=1e-10, vary=False)
pvic_params['R'].set(value=0, vary=False)
result = pvic_model.fit(data=Yfitdata, params=pvic_params, x=X)
print(result.fit_report())
plot_fit_result(X, Yfitdata, result)
freeParams = [k for k in list(pvic_params.keys()) if pvic_params[k].vary==True]
plt.title(f"{len(freeParams)} free parameters: {freeParams}")

[[Model]]
    Model(pVIC)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 105
    # data points      = 120
    # variables        = 5
    chi-square         = 3.3204e+11
    reduced chi-square = 2.8873e+09
    Akaike info crit   = 2618.92398
    Bayesian info crit = 2632.86144
[[Variables]]
    A:      139800.970 +/- 832.210128 (0.60%) (init = 200000)
    alpha:  130.269301 +/- 1.82009419 (1.40%) (init = 140)
    beta:   1e-10 (fixed)
    R:      0 (fixed)
    gamma:  1.0779e-07 +/- 1.4438e-04 (133947.70%) (init = 0.001)
    sigma:  0.00596479 +/- 2.2440e-04 (3.76%) (init = 0.0066)
    k:      0.05 (fixed)
    x0:    -8.01731281 +/- 2.1916e-04 (0.00%) (init = -8)
[[Correlations]] (unreported correlations are < 0.100)
    C(alpha, x0)    =  0.942
    C(alpha, sigma) =  0.641
    C(sigma, x0)    =  0.631
    C(A, gamma)     =  0.472
    C(gamma, sigma) = -0.366
    C(A, x0)        = -0.227
    C(alpha, gamma) =  0.219
    C(A, alpha)     = -0.212
    C(A, sig

Text(0.5, 1.0, "5 free parameters: ['A', 'alpha', 'gamma', 'sigma', 'x0']")

## Batch fit data above Hc 
to determine the values of shared parameters

#### Next steps as of 2020-04-21
* Update code for each step of the (10 10 0) peak fitting procedure to replace each repeated section of code by the corresponding function
* Plot fit at each step to check on the quality of the fit
* If there is no quick and easy way of getting a good fit, move on to the (8 8 0) peak

### Batch fitting functions

In [198]:
def xyBatchFitNData(nData, num_spec):
    num_spec # number of datasets to include in fit
    spec_rng = range(len(nData)-num_spec, len(nData))
    X = np.stack([nData.spectra[idx].hh0[data_select] for idx in spec_rng])
    # x data for batch fitting
    Y = np.stack([nData.spectra[idx].Inorm[data_select] for idx in spec_rng])
    return X, Y

In [194]:
def xFitInitParams(fitParams, refParams, resultParams=None):
    """
    Initialize parameters for the next fit using the results of the previous fit
    and, if necessary, the default values of a reference set of parameters
    """
    # For those parameters that have been computed in the last run, 
    # use as initial values for the next run the best fit values obtained in the last
    if resultParams is not None:
        for key in fitParams.keys():
            try:
                fitParams[key].value = resultParams[key].value
            except KeyError: # in case fitParams has been modified since last fitting run
                continue

    # Create additional fit parameters, e.g. if the number of datasets has been extended
    for spec_idx, _ in enumerate(X):
        for k, v in refParams.items():
            if k in ['A', 'xp']:
            # fit parameters that are different for each dataset are assigned individual names
                par_key = f'{k}{spec_rng[spec_idx]}'
                if par_key not in fitParams.keys():
                    fitParams.add( par_key, value=refParams[k].value, 
                                      min=refParams[k].min, vary=refParams[k].vary )
            elif resultParams is None: # if there are no shared fit parameters, because no fit has been performed yet
            # all other parameters are shared by all datasets and are assigned the "generic" name from refParams
                fitParams[k] = refParams[k]
    return fitParams

### Batch fitting of single (10 10 0) peak spectra with single pVIC function
above Hc

This will help fix the value of the parameters shared by all spectra, i.e. alpha, beta, R, gamma and sigma

#### Dataset selection

In [152]:
peak_center = -10.0# center of unsplit peak to be studied in the following, in reciprocal space units
npeak_interval = .15# half of plot interval
data_select = np.logical_and(nData.spectra[dat_idx].hh0 > peak_center - npeak_interval, 
                             nData.spectra[dat_idx].hh0 < peak_center + npeak_interval)

#### Create fit parameters and specify their properties
including initial values, constraints, etc.
Note: when all pVIC parameters (except k) are free, the fit is very sensitive to initial values and can easily produce NaN numbers.
The following initial parameters appear to yield a result without raising any error:
A=2e5, alpha=140, beta=1e-3, R=1e-3, gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center

In [157]:
pvic_params = pvic_model.make_params(A=2e5, alpha=140, beta=1e-3, R=1e-3, 
                                     gamma=1e-3, sigma=6.6e-3, k=.05, xp=peak_center)
for k in pvic_params.keys():
    pvic_params[k].set(min=0, vary=True)
pvic_params['xp'].set(min=-np.inf)
pvic_params['k'].vary = False
pvic_params

name,value,initial value,min,max,vary
A,200000.000,None,0.00000000,inf,True
alpha,140.000000,None,0.00000000,inf,True
beta,1.0000e-03,None,0.00000000,inf,True
R,1.0000e-03,None,0.00000000,inf,True
gamma,1.0000e-03,None,0.00000000,inf,True
sigma,0.00660000,None,0.00000000,inf,True
k,0.05000000,None,0.00000000,inf,False
xp,-10.0000000,None,-inf,inf,True


#### Fit with 2 independent free parameters per dataset and 5 free parameters shared between all datasets

In [158]:
num_spec = 10 # number of datasets to include in fit
spec_rng = range(len(nData)-num_spec, len(nData))
X = np.stack([nData.spectra[idx].hh0[data_select] for idx in spec_rng])
# x data for batch fitting
Yfitdata = np.stack([nData.spectra[idx].Inorm[data_select] for idx in spec_rng])

### Initialize parameters 
xpvic_params = Parameters()
# create new set of parameters for each dataset to fit
for spec_idx, _ in enumerate(X):
    for k, v in pvic_params.items():
        if k in ['A', 'xp']:# fit parameters that are different for each dataset
            par_key = f'{k}{spec_rng[spec_idx]}'
            xpvic_params.add( par_key, value=pvic_params[k].value, 
                              min=pvic_params[k].min, vary=pvic_params[k].vary )
        else:# all other parameters are shared by all datasets and are assigned the "generic" name from pvic_params
            xpvic_params[k] = pvic_params[k]

### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(X, Yfitdata, spec_rng))

In [159]:
# report_fit(result)
result.params

name,value,standard error,relative error,initial value,min,max,vary
A92,236583.701,1562.04935,(0.66%),200000.0,0.00000000,inf,True
alpha,103.917430,0.56892793,(0.55%),140,0.00000000,inf,True
beta,1882.66221,1039629.92,(55221.27%),0.001,0.00000000,inf,True
R,9.7323e-04,1.11093368,(114148.78%),0.001,0.00000000,inf,True
gamma,0.00140661,1.3707e-04,(9.74%),0.001,0.00000000,inf,True
sigma,0.00350110,2.3479e-04,(6.71%),0.0066,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp92,-10.0220116,5.7907e-04,(0.01%),-10.0,-inf,inf,True
A93,231794.760,1559.88728,(0.67%),200000.0,0.00000000,inf,True
xp93,-10.0221229,5.8667e-04,(0.01%),-10.0,-inf,inf,True


#### Fit with 2 independent free parameters per dataset and 3 free parameters shared between all datasets
The huge variability on R and beta seen when leaving those free implies that they play little role on the fit, so let's fix them both to zero.
Actually, beta=0 generates NaN data (not sure why) so let's fix it to 1e-10

In [185]:
num_spec = 10 # number of datasets to include in fit
spec_rng = range(len(nData)-num_spec, len(nData))
X = np.stack([nData.spectra[idx].hh0[data_select] for idx in spec_rng])
# x data for batch fitting
Yfitdata = np.stack([nData.spectra[idx].Inorm[data_select] for idx in spec_rng])

### Initialize parameters using results of previous fit
for key in xpvic_params.keys():
    xpvic_params[key].value = result.params[key].value
for key in ['beta', 'R']:
    xpvic_params[key].vary = False

xpvic_params['R'].value = 0
xpvic_params['beta'].value = 1e-10
    
### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(X, Yfitdata, spec_rng))

In [186]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A92,236582.861,1487.03650,(0.63%),236582.86051030117,0.00000000,inf,True
alpha,103.917826,0.56246575,(0.54%),103.91782644306636,0.00000000,inf,True
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
R,0.00000000,0.00000000,,0,0.00000000,inf,False
gamma,0.00140644,1.3646e-04,(9.70%),0.0014064403325111385,0.00000000,inf,True
sigma,0.00350177,2.2494e-04,(6.42%),0.003501766791173866,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp92,-10.0220111,1.4390e-04,(0.00%),-10.022011057730563,-inf,inf,True
A93,231793.965,1480.09345,(0.64%),231793.96506228583,0.00000000,inf,True
xp93,-10.0221223,1.4564e-04,(0.00%),-10.022122347339756,-inf,inf,True


#### Same but fitting 30 datasets
significantly reduces the error on gama and corroborates the values of alpha and sigma

In [191]:
num_spec = 30 # number of datasets to include in fit
spec_rng = range(len(nData)-num_spec, len(nData))
X = np.stack([nData.spectra[idx].hh0[data_select] for idx in spec_rng])
# x data for batch fitting
Yfitdata = np.stack([nData.spectra[idx].Inorm[data_select] for idx in spec_rng])

### Initialize parameters 
for key in xpvic_params.keys():
    try:
        xpvic_params[key].value = result.params[key].value
    except KeyError: # in case xpvic_params has been modified since last fitting run
        continue

for spec_idx, _ in enumerate(X):
    for k, v in pvic_params.items():
        if k in ['A', 'xp']:
        # fit parameters that are different for each dataset are assigned individual names
            par_key = f'{k}{spec_rng[spec_idx]}'
            if par_key not in xpvic_params.keys():
                xpvic_params.add( par_key, value=pvic_params[k].value, 
                                  min=pvic_params[k].min, vary=pvic_params[k].vary )

for key in ['beta', 'R']:
    xpvic_params[key].vary = False

xpvic_params['R'].value = 0
xpvic_params['beta'].value = 1e-10

### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(X, Yfitdata, spec_rng))

In [192]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A72,257816.939,1501.83081,(0.58%),200000.0,0.00000000,inf,True
alpha,106.001932,0.35730928,(0.34%),103.91782644306636,0.00000000,inf,True
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
R,0.00000000,0.00000000,,0,0.00000000,inf,False
gamma,0.00326599,5.7844e-05,(1.77%),0.0014064403325111385,0.00000000,inf,True
sigma,3.0512e-06,5.5147e-06,(180.74%),0.003501766791173866,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp72,-10.0217079,1.3052e-04,(0.00%),-10.0,-inf,inf,True
A73,261708.320,1502.90364,(0.57%),200000.0,0.00000000,inf,True
xp73,-10.0213369,1.2898e-04,(0.00%),-10.0,-inf,inf,True


#### Fit with 2 independent free parameters per dataset and 2 free parameters shared between all datasets
Fix alpha to the value obtained from previous fit

In [195]:
Xfitdata, Yfitdata = xyBatchFitNData(nData, num_spec=30)

xpvic_params = xFitInitParams(xpvic_params, pvic_params, result.params)

xpvic_params['alpha'].vary = False

### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(Xfitdata, Yfitdata, spec_rng))

In [196]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A72,257816.939,1497.40915,(0.58%),257816.93942249773,0.00000000,inf,True
alpha,106.001932,0.00000000,(0.00%),106.00193215253873,0.00000000,inf,False
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
R,0.00000000,0.00000000,,0,0.00000000,inf,False
gamma,0.00326599,3.2993e-05,(1.01%),0.003265994624882751,0.00000000,inf,True
sigma,3.0512e-06,5.9187e-06,(193.98%),3.0511627535645403e-06,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp72,-10.0217079,1.1706e-04,(0.00%),-10.02170791372692,-inf,inf,True
A73,261708.320,1498.35305,(0.57%),261708.32038426344,0.00000000,inf,True
xp73,-10.0213369,1.1534e-04,(0.00%),-10.021336850511767,-inf,inf,True


#### Fit with 2 independent free parameters per dataset and 1 free parameter shared between all datasets
Fix gamma to the value obtained from the previous fit

In [199]:
Xfitdata, Yfitdata = xyBatchFitNData(nData, num_spec=30)

xpvic_params = xFitInitParams(xpvic_params, pvic_params, result.params)

xpvic_params['gamma'].vary = False

### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(X, Yfitdata, spec_rng))

In [200]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
A72,257816.939,1465.08507,(0.57%),257816.93942249773,0.00000000,inf,True
alpha,106.001932,0.00000000,(0.00%),106.00193215253873,0.00000000,inf,False
beta,1.0000e-10,0.00000000,(0.00%),1e-10,0.00000000,inf,False
R,0.00000000,0.00000000,,0,0.00000000,inf,False
gamma,0.00326599,0.00000000,(0.00%),0.003265994624882751,0.00000000,inf,False
sigma,3.0512e-06,5.9176e-06,(193.94%),3.0511627535645403e-06,0.00000000,inf,True
k,0.05000000,0.00000000,(0.00%),0.05,0.00000000,inf,False
xp72,-10.0217079,1.1704e-04,(0.00%),-10.02170791372692,-inf,inf,True
A73,261708.320,1465.05597,(0.56%),261708.32038426344,0.00000000,inf,True
xp73,-10.0213369,1.1532e-04,(0.00%),-10.021336850511767,-inf,inf,True


#### Fit with 2 independent free parameters per dataset and all parameters shared between datasets fixed
Fix gamma to 5.3e-4, which is the value obtained from the previous fit

In [203]:
Xfitdata, Yfitdata = xyBatchFitNData(nData, num_spec=30)

xpvic_params = xFitInitParams(xpvic_params, pvic_params, result.params)

xpvic_params['sigma'].vary = False

### Perform fit 
result = minimize(xpVIC_residual, xpvic_params, args=(X, Yfitdata, spec_rng))

In [204]:
result

#### Plot fit result

In [205]:
bestparams = np.zeros((len(spec_rng),8))# shape = # of datasets x # of parameters in fit function (pVIC)
for spec_idx in range(len(spec_rng)):
    for par_idx, k in enumerate(pvic_params.keys()):
        par_key = f'{k}{spec_rng[spec_idx]}' 
        # parameter name is a concatenation of the generic parameter name,
        # as defined in the pVIC function, and the spectrum index
        try:
            bestparams[spec_idx][par_idx] = result.params[par_key].value
        except KeyError:
            bestparams[spec_idx][par_idx] = result.params[k].value
        
# list(result.params.valuesdict().values())
# bestparams

In [206]:
fig = plt.figure()
for idx, spec_idx in enumerate(spec_rng):
    bestfit = pVIC(X[idx], *bestparams[idx][:8])
    plt.plot(X[idx], fitdata[idx], 'o', label=f"expt {nData['H (T)'][spec_idx]:.2f}T")
    plt.plot(X[idx], bestfit, '-', label=f"fit {nData['H (T)'][spec_idx]:.2f}T")
    plt.legend(loc='best')
    plt.show()
freeParams = [k for k in list(xpvic_params.keys()) if xpvic_params[k].vary==True]
plt.title(f"TmVO$_4$ neutrons {len(freeParams)} free parameters")

Text(0.5, 1.0, 'TmVO$_4$ neutrons 60 free parameters')

### Batch fitting of single (8 8 0) peak spectra with single pVIC function
above Hc

This will help fix the value of the parameters shared by all spectra, i.e. alpha, beta, R, gamma and sigma